# Το Πρόβλημα του Περιοδεύοντος Πωλητή (Travelling Salesman Problem, TSP)  με χρήση γενετικών αλγορίθμων

Είναι ένα κλασσικό συνδιαστικό πρόβλημα βελτιστοποίησης. 
Ένας πωλητής πρέπει να επισκεφτεί n πελάτες σε διαφορετικές πόλεις. 
Το ζητούμενο είναι να βρεθεί η σύντομη η/και χαμηλότερη κόστους πορεία την οποία περνά μία μόνο φορά απο κάθε πόλη και επιστρέφει στην πόλη στην οποία ξεκίνησε ο Πωλητής.

Οι Γενετικοί αλγόριθμοι ειναι μια μέθοδος αναζήτησης βέλτιστων λύσεων σε συστήματα που μπορούν να περιγραφούν ως μαθηματικό πρόβλημα.
Ο τρόπος λειτουργίας τους είναι εμπνευσμένος απο την βιολογία. Χρησιμοποιεί την ιδέα της εξέλιξης των ειδών μέσω γενετικής μετάλλαξης, φυσικής επιλογής και διασταύρωσης.

## Περίληψη Εργασίας
Σκοπός είναι να μεγιστοποιήσουμε τη συνάρτηση καταλληλότητας και να βρούμε την καλύτερη λύση.
Οπότε παριστάνουμε την κάθε πόλη με ένα κόμβο και τοποθετούμε μία ακμή ανάμεσα σε δύο πόλεις εάν συνδέονται άμεσα. Σχηματίζεται ο πληθυσμός τυχαία, αξιολογείται η κάθε λύση ξεχωριστά και περνάμε στην επόμενη γενία αφού εφαρμοστούν οι γενετικοί τελεστές. Έτσι κάποιες λύσεις πάνε στην επόμενη γενία αυτούσιες με τη διαδικασία της επιλογής (selection), κάποιες θα υποστούν μετάλλαξη (mutation) και κάποιες άλλες θα διασταυρωθούν (crossover) μεταξύ τους δημιουργώντας απογόνους.


## Δομή ενός ΓΑ

1. Αρχικοποίση - Δημιουργία αρχικού πληθυσμόυ(Initialization)
2. Αποκωδικοποίηση(Decoding)
3. Υπολογισμός ικανότητας η αξιολόγησης(Fitness calculation or evaluation)
4. Αναπαραγωγή(Reproduction)
    I. Επιλογή(Selection)
    II. Διασταύρωση(Crossover)
    III. Μετάλλαξη(Mutation)
5. Επανάληψη από το βήμα (2) μέχρι να ικανοποιηθούν τα κριτήρια τερματισμού αλγορίθμου.

## Υλοποίηση